In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import f_regression, SelectKBest, RFE
import os
from sklearn.impute import SimpleImputer
from env import get_db_url
import warnings
warnings.filterwarnings('ignore')
import math 

In [2]:
def acquire_zillow():
    '''
    This function checks for a copy of the dataset in the local directory 
    and pulls a new copy and saves it if there is not one,
    it then cleans the data by removing significant outliers then
    removing the rows with null values for 'yearbuilt'
    '''
    #assign the file name
    filename = 'zillow4.csv'
    #check if the file exists in the current directory and read it if it is
    if os.path.exists(filename):
        print('Reading from csv file...')
        #read the local .csv into the notebook
        df = pd.read_csv(filename)
        return df
    #assign the sql query to a variable for use in pulling a new copy of the dataset from the database
    query = '''
    SELECT pro.bedroomcnt, pro.bathroomcnt, pro.calculatedfinishedsquarefeet, 
    pro.taxvaluedollarcnt, pro.yearbuilt, pro.fips
    FROM properties_2017 AS pro
    JOIN predictions_2017 AS pre USING(parcelid)
    WHERE pro.propertylandusetypeid = 261;
    '''
    #if needed pull a fresh copy of the dataset from the database
    print('Getting a fresh copy from SQL database...')
    df = pd.read_sql(query, get_db_url('zillow'))
    #save a copy of the dataset to the local directory as a .csv file
    df.to_csv(filename, index=False)
    return df

In [3]:
df = acquire_zillow()
df.head()

Reading from csv file...


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,fips
0,4.0,3.5,3100.0,1023282.0,1998.0,6059.0
1,2.0,1.0,1465.0,464000.0,1967.0,6111.0
2,3.0,2.0,1243.0,564778.0,1962.0,6059.0
3,4.0,3.0,2376.0,145143.0,1970.0,6037.0
4,4.0,3.0,2962.0,773303.0,1950.0,6037.0


In [4]:
df.shape

(52442, 6)

In [5]:
df.isnull().sum()

bedroomcnt                        0
bathroomcnt                       0
calculatedfinishedsquarefeet     82
taxvaluedollarcnt                 1
yearbuilt                       116
fips                              0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
fips                            0
dtype: int64

In [8]:
df.shape

(52320, 6)

In [9]:
df.fips.value_counts()

6037.0    33880
6059.0    14082
6111.0     4358
Name: fips, dtype: int64

In [10]:
df1 = df[df['fips']==6037.0]
df2 = df[df['fips']==6059.0]
df3 = df[df['fips']==6111.0]

---

In [11]:
def split_data(df):
    train_val, test = train_test_split(df, train_size = 0.8, random_state=123)
    train, validate = train_test_split(train_val, train_size = 0.7, random_state=123)
    return train, validate, test

In [12]:
def remove_outliers(df, k, col_list):
    ''' this function will remove outliers from a list of columns in a dataframe 
        and return that dataframe. A list of columns with significant outliers is 
        assigned to a variable in the below wrangle function and can be modified if needed
    '''
    #loop throught the columns in the list
    for col in col_list:
        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        iqr = q3 - q1   # calculate interquartile range
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound
        # return dataframe without outliers
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)] 
    return df

---

**DF1**

In [16]:
train, validate, test = split_data(df1)
train.shape, validate.shape, test.shape

((18972, 6), (8132, 6), (6776, 6))

In [17]:
col_list = ['bedroomcnt', 'bathroomcnt','calculatedfinishedsquarefeet']
train = remove_outliers(train, 1.5, col_list)
train.shape

(17266, 6)

In [18]:
x_train = train[['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'yearbuilt']]
y_train = train[['taxvaluedollarcnt']]
x_train.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
37118,2.0,1.0,1120.0,1942.0
8181,3.0,2.0,1261.0,1952.0
29778,4.0,2.0,1103.0,1985.0
40880,3.0,2.0,1712.0,1952.0
35077,3.0,2.0,1900.0,1959.0


In [19]:
#create a min-max scaler object
scaler = MinMaxScaler()
#fit the scaler with the train data
scaler.fit(x_train)
#output the results of the scaler into a new df to use with feature selection
x_train_scaled = scaler.transform(x_train)
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_train_scaled_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
37118,0.000000,0.000000,0.262566,0.463768
8181,0.333333,0.333333,0.309193,0.536232
29778,0.666667,0.333333,0.256944,0.775362
40880,0.333333,0.333333,0.458333,0.536232
35077,0.333333,0.333333,0.520503,0.586957


In [20]:
x_validate = validate[['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'yearbuilt']]
y_validate = validate[['taxvaluedollarcnt']]
x_validate.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
4763,2.0,2.0,1132.0,1921.0
41771,2.0,2.0,1055.0,1988.0
2789,4.0,7.0,12039.0,1992.0
30297,3.0,3.0,1637.0,1989.0
18920,3.0,3.0,1406.0,1994.0


In [21]:
x_validate_scaled = scaler.transform(x_validate)
x_validate_scaled_df = pd.DataFrame(x_validate_scaled, columns=x_train.columns, index=x_validate.index)
x_validate_scaled_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
4763,0.000000,0.333333,0.266534,0.311594
41771,0.000000,0.333333,0.241071,0.797101
2789,0.666667,2.000000,3.873347,0.826087
30297,0.333333,0.666667,0.433532,0.804348
18920,0.333333,0.666667,0.357143,0.840580


In [28]:
x = x_train_scaled_df
y = y_train
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2).fit(x, y)
print('Top 2 features according to RFE:')
x.columns[rfe.get_support()]

Top 2 features according to RFE:


Index(['calculatedfinishedsquarefeet', 'yearbuilt'], dtype='object')

In [29]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=x.columns)

,rfe_ranking
bedroomcnt,3
bathroomcnt,2
calculatedfinishedsquarefeet,1
yearbuilt,1


---

In [22]:
predictions1 = pd.DataFrame({'actual': validate.taxvaluedollarcnt})
predictions1.head()

,actual
4763,253246.0
41771,74943.0
2789,3794515.0
30297,103583.0
18920,226706.0


In [23]:
predictions1['baseline'] = train.taxvaluedollarcnt.mean()
predictions1.head()

,actual,baseline
4763,253246.0,405669.031507
41771,74943.0,405669.031507
2789,3794515.0,405669.031507
30297,103583.0,405669.031507
18920,226706.0,405669.031507


In [24]:
x_train_poly_cols = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
x_val_poly_cols = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(x_train_poly_cols)
x_train_poly = pd.DataFrame(
    poly.transform(x_train_poly_cols),
    columns=poly.get_feature_names(x_train_poly_cols.columns),
    index=x_train_poly_cols.index,
)

poly_rm = LinearRegression()
poly_rm.fit(x_train_poly, y_train)

x_validate_poly = poly.transform(x_val_poly_cols)
predictions1['polynomial3_intsonly'] = poly_rm.predict(x_validate_poly)
predictions1.head()

,actual,baseline,polynomial3_intsonly
4763,253246.0,405669.031507,3.551383e+05
41771,74943.0,405669.031507,1.781230e+05
2789,3794515.0,405669.031507,7.349323e+06
30297,103583.0,405669.031507,3.101912e+05
18920,226706.0,405669.031507,2.306313e+05


In [25]:
x_train_poly_cols = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
x_val_poly_cols = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(x_train_poly_cols)
x_train_poly = pd.DataFrame(
    poly.transform(x_train_poly_cols),
    columns=poly.get_feature_names(x_train_poly_cols.columns),
    index=x_train_poly_cols.index,
)

poly_rm = LinearRegression()
poly_rm.fit(x_train_poly, y_train)

x_validate_poly = poly.transform(x_val_poly_cols)
predictions1['polynomial4_intsonly'] = poly_rm.predict(x_validate_poly)
predictions1.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly
4763,253246.0,405669.031507,3.551383e+05,4.144775e+05
41771,74943.0,405669.031507,1.781230e+05,2.492511e+05
2789,3794515.0,405669.031507,7.349323e+06,8.859314e+06
30297,103583.0,405669.031507,3.101912e+05,3.571889e+05
18920,226706.0,405669.031507,2.306313e+05,2.642259e+05


In [31]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt']]
lm = LinearRegression()
lm.fit(x, y_train)
predictions1['lm_top2x'] = lm.predict(z)
predictions1.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x
4763,253246.0,405669.031507,3.551383e+05,4.144775e+05,3.667042e+05
41771,74943.0,405669.031507,1.781230e+05,2.492511e+05,1.097986e+05
2789,3794515.0,405669.031507,7.349323e+06,8.859314e+06,3.569803e+06
30297,103583.0,405669.031507,3.101912e+05,3.571889e+05,2.903956e+05
18920,226706.0,405669.031507,2.306313e+05,2.642259e+05,1.999830e+05


In [33]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
lm2 = LinearRegression()
lm2.fit(x, y_train)
predictions1['lm_top3x'] = lm2.predict(z)
predictions1.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x,lm_top3x
4763,253246.0,405669.031507,3.551383e+05,4.144775e+05,3.667042e+05,4.186643e+05
41771,74943.0,405669.031507,1.781230e+05,2.492511e+05,1.097986e+05,1.287066e+05
2789,3794515.0,405669.031507,7.349323e+06,8.859314e+06,3.569803e+06,3.218636e+06
30297,103583.0,405669.031507,3.101912e+05,3.571889e+05,2.903956e+05,3.436990e+05
18920,226706.0,405669.031507,2.306313e+05,2.642259e+05,1.999830e+05,2.659582e+05


In [36]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
lm3 = LinearRegression()
lm3.fit(x, y_train)
predictions1['lm_allx'] = lm3.predict(z)
predictions1.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x,lm_top3x,lm_allx
4763,253246.0,405669.031507,3.551383e+05,4.144775e+05,3.667042e+05,4.186643e+05,4.841431e+05
41771,74943.0,405669.031507,1.781230e+05,2.492511e+05,1.097986e+05,1.287066e+05,2.113874e+05
2789,3794515.0,405669.031507,7.349323e+06,8.859314e+06,3.569803e+06,3.218636e+06,3.709355e+06
30297,103583.0,405669.031507,3.101912e+05,3.571889e+05,2.903956e+05,3.436990e+05,3.853063e+05
18920,226706.0,405669.031507,2.306313e+05,2.642259e+05,1.999830e+05,2.659582e+05,2.991310e+05


In [37]:
def calculate_mse(y_predicted):
    return mean_squared_error(predictions1.actual, y_predicted)

predictions1.apply(calculate_mse)

actual                  0.000000e+00
baseline                8.883195e+11
polynomial3_intsonly    3.488599e+11
polynomial4_intsonly    3.479130e+11
lm_top2x                6.248579e+11
lm_top3x                6.237363e+11
lm_allx                 6.098242e+11
dtype: float64

In [38]:
predictions1.apply(calculate_mse).apply(math.sqrt)

actual                       0.000000
baseline                942507.047615
polynomial3_intsonly    590643.648763
polynomial4_intsonly    589841.530967
lm_top2x                790479.565070
lm_top3x                789769.757598
lm_allx                 780912.426966
dtype: float64

---

**DF2**

In [13]:
train, validate, test = split_data(df2)
train.shape, validate.shape, test.shape

((7885, 6), (3380, 6), (2817, 6))

In [14]:
col_list = ['bedroomcnt', 'bathroomcnt','calculatedfinishedsquarefeet']
train = remove_outliers(train, 1.5, col_list)
train.shape

(7168, 6)

In [15]:
x_train = train[['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'yearbuilt']]
y_train = train[['taxvaluedollarcnt']]
x_train.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
6981,4.0,2.5,2578.0,1966.0
39219,4.0,2.5,2403.0,1978.0
34413,4.0,2.5,2710.0,1971.0
45652,4.0,1.5,1508.0,1965.0
19135,2.0,1.0,1010.0,1949.0


In [16]:
#create a min-max scaler object
scaler = MinMaxScaler()
#fit the scaler with the train data
scaler.fit(x_train)
#output the results of the scaler into a new df to use with feature selection
x_train_scaled = scaler.transform(x_train)
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_train_scaled_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
6981,0.666667,0.500000,0.606367,0.591667
39219,0.666667,0.500000,0.554299,0.691667
34413,0.666667,0.500000,0.645641,0.633333
45652,0.666667,0.166667,0.288010,0.583333
19135,0.000000,0.000000,0.139839,0.450000


In [17]:
x_validate = validate[['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'yearbuilt']]
y_validate = validate[['taxvaluedollarcnt']]
x_validate.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
34189,3.0,2.0,1434.0,1962.0
16063,3.0,3.0,3530.0,1987.0
19954,5.0,5.0,5525.0,1991.0
10163,4.0,3.0,2501.0,2006.0
43690,3.0,3.0,1996.0,1997.0


In [18]:
x_validate_scaled = scaler.transform(x_validate)
x_validate_scaled_df = pd.DataFrame(x_validate_scaled, columns=x_train.columns, index=x_validate.index)
x_validate_scaled_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
34189,0.333333,0.333333,0.265992,0.558333
16063,0.333333,0.666667,0.889616,0.766667
19954,1.000000,1.333333,1.483190,0.800000
10163,0.666667,0.666667,0.583457,0.925000
43690,0.333333,0.666667,0.433204,0.850000


In [19]:
x = x_train_scaled_df
y = y_train
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2).fit(x, y)
print('Top 2 features according to RFE:')
x.columns[rfe.get_support()]

Top 2 features according to RFE:


Index(['calculatedfinishedsquarefeet', 'yearbuilt'], dtype='object')

In [20]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=x.columns)

,rfe_ranking
bedroomcnt,2
bathroomcnt,3
calculatedfinishedsquarefeet,1
yearbuilt,1


In [21]:
predictions2 = pd.DataFrame({'actual': validate.taxvaluedollarcnt})
predictions2.head()

,actual
34189,60126.0
16063,1186992.0
19954,1453151.0
10163,973038.0
43690,771369.0


In [22]:
predictions2['baseline'] = train.taxvaluedollarcnt.mean()
predictions2.head()

,actual,baseline
34189,60126.0,506558.233677
16063,1186992.0,506558.233677
19954,1453151.0,506558.233677
10163,973038.0,506558.233677
43690,771369.0,506558.233677


In [23]:
x_train_poly_cols = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
x_val_poly_cols = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(x_train_poly_cols)
x_train_poly = pd.DataFrame(
    poly.transform(x_train_poly_cols),
    columns=poly.get_feature_names(x_train_poly_cols.columns),
    index=x_train_poly_cols.index,
)

poly_rm = LinearRegression()
poly_rm.fit(x_train_poly, y_train)

x_validate_poly = poly.transform(x_val_poly_cols)
predictions2['polynomial3_intsonly'] = poly_rm.predict(x_validate_poly)
predictions2.head()

,actual,baseline,polynomial3_intsonly
34189,60126.0,506558.233677,3.464392e+05
16063,1186992.0,506558.233677,9.698520e+05
19954,1453151.0,506558.233677,2.308406e+06
10163,973038.0,506558.233677,7.848918e+05
43690,771369.0,506558.233677,6.105215e+05


In [24]:
x_train_poly_cols = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
x_val_poly_cols = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(x_train_poly_cols)
x_train_poly = pd.DataFrame(
    poly.transform(x_train_poly_cols),
    columns=poly.get_feature_names(x_train_poly_cols.columns),
    index=x_train_poly_cols.index,
)

poly_rm = LinearRegression()
poly_rm.fit(x_train_poly, y_train)

x_validate_poly = poly.transform(x_val_poly_cols)
predictions2['polynomial4_intsonly'] = poly_rm.predict(x_validate_poly)
predictions2.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly
34189,60126.0,506558.233677,3.464392e+05,3.589945e+05
16063,1186992.0,506558.233677,9.698520e+05,1.114274e+06
19954,1453151.0,506558.233677,2.308406e+06,2.279556e+06
10163,973038.0,506558.233677,7.848918e+05,7.524852e+05
43690,771369.0,506558.233677,6.105215e+05,6.394763e+05


In [26]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt']]
lm = LinearRegression()
lm.fit(x, y_train)
predictions2['lm_top2x'] = lm.predict(z)
predictions2.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x
34189,60126.0,506558.233677,3.464392e+05,3.589945e+05,3.517559e+05
16063,1186992.0,506558.233677,9.698520e+05,1.114274e+06,9.553564e+05
19954,1453151.0,506558.233677,2.308406e+06,2.279556e+06,1.470604e+06
10163,973038.0,506558.233677,7.848918e+05,7.524852e+05,7.526607e+05
43690,771369.0,506558.233677,6.105215e+05,6.394763e+05,5.983201e+05


In [27]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
lm2 = LinearRegression()
lm2.fit(x, y_train)
predictions2['lm_top3x'] = lm2.predict(z)
predictions2.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x,lm_top3x
34189,60126.0,506558.233677,3.464392e+05,3.589945e+05,3.517559e+05,3.538341e+05
16063,1186992.0,506558.233677,9.698520e+05,1.114274e+06,9.553564e+05,9.403126e+05
19954,1453151.0,506558.233677,2.308406e+06,2.279556e+06,1.470604e+06,1.479411e+06
10163,973038.0,506558.233677,7.848918e+05,7.524852e+05,7.526607e+05,7.522925e+05
43690,771369.0,506558.233677,6.105215e+05,6.394763e+05,5.983201e+05,6.108191e+05


In [28]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
lm3 = LinearRegression()
lm3.fit(x, y_train)
predictions2['lm_allx'] = lm3.predict(z)
predictions2.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x,lm_top3x,lm_allx
34189,60126.0,506558.233677,3.464392e+05,3.589945e+05,3.517559e+05,3.538341e+05,3.646503e+05
16063,1186992.0,506558.233677,9.698520e+05,1.114274e+06,9.553564e+05,9.403126e+05,1.042937e+06
19954,1453151.0,506558.233677,2.308406e+06,2.279556e+06,1.470604e+06,1.479411e+06,1.549444e+06
10163,973038.0,506558.233677,7.848918e+05,7.524852e+05,7.526607e+05,7.522925e+05,7.181793e+05
43690,771369.0,506558.233677,6.105215e+05,6.394763e+05,5.983201e+05,6.108191e+05,6.449884e+05


In [29]:
def calculate_mse(y_predicted):
    return mean_squared_error(predictions2.actual, y_predicted)

predictions2.apply(calculate_mse)

actual                  0.000000e+00
baseline                3.620621e+11
polynomial3_intsonly    1.707587e+11
polynomial4_intsonly    1.662254e+11
lm_top2x                2.031236e+11
lm_top3x                2.017384e+11
lm_allx                 1.914123e+11
dtype: float64

In [30]:
predictions2.apply(calculate_mse).apply(math.sqrt)

actual                       0.000000
baseline                601715.993045
polynomial3_intsonly    413229.554007
polynomial4_intsonly    407707.437513
lm_top2x                450692.310615
lm_top3x                449152.985274
lm_allx                 437506.942503
dtype: float64

---

**DF3**

In [13]:
train, validate, test = split_data(df3)
train.shape, validate.shape, test.shape

((2440, 6), (1046, 6), (872, 6))

In [14]:
col_list = ['bedroomcnt', 'bathroomcnt','calculatedfinishedsquarefeet']
train = remove_outliers(train, 1.5, col_list)
train.shape

(2239, 6)

In [15]:
x_train = train[['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'yearbuilt']]
y_train = train[['taxvaluedollarcnt']]
x_train.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
43436,3.0,1.5,1376.0,1962.0
2018,4.0,2.0,1701.0,1965.0
27370,3.0,2.0,1160.0,1973.0
46312,4.0,2.0,1956.0,1964.0
13484,4.0,1.5,1145.0,1962.0


In [16]:
#create a min-max scaler object
scaler = MinMaxScaler()
#fit the scaler with the train data
scaler.fit(x_train)
#output the results of the scaler into a new df to use with feature selection
x_train_scaled = scaler.transform(x_train)
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_train_scaled_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
43436,0.333333,0.166667,0.273171,0.582677
2018,0.666667,0.333333,0.372256,0.606299
27370,0.333333,0.333333,0.207317,0.669291
46312,0.666667,0.333333,0.450000,0.598425
13484,0.666667,0.166667,0.202744,0.582677


In [17]:
x_validate = validate[['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'yearbuilt']]
y_validate = validate[['taxvaluedollarcnt']]
x_validate.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
13404,5.0,4.5,3622.0,2006.0
2305,4.0,4.5,4007.0,2000.0
11329,6.0,7.0,7770.0,2004.0
9365,3.0,3.0,1894.0,1959.0
7841,4.0,2.0,1961.0,1962.0


In [18]:
x_validate_scaled = scaler.transform(x_validate)
x_validate_scaled_df = pd.DataFrame(x_validate_scaled, columns=x_train.columns, index=x_validate.index)
x_validate_scaled_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt
13404,1.000000,1.166667,0.957927,0.929134
2305,0.666667,1.166667,1.075305,0.881890
11329,1.333333,2.000000,2.222561,0.913386
9365,0.333333,0.666667,0.431098,0.559055
7841,0.666667,0.333333,0.451524,0.582677


In [19]:
x = x_train_scaled_df
y = y_train
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2).fit(x, y)
print('Top 2 features according to RFE:')
x.columns[rfe.get_support()]

Top 2 features according to RFE:


Index(['calculatedfinishedsquarefeet', 'yearbuilt'], dtype='object')

In [20]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=x.columns)

,rfe_ranking
bedroomcnt,2
bathroomcnt,3
calculatedfinishedsquarefeet,1
yearbuilt,1


In [21]:
predictions3 = pd.DataFrame({'actual': validate.taxvaluedollarcnt})
predictions3.head()

,actual
13404,849000.0
2305,1298990.0
11329,1636139.0
9365,302961.0
7841,376973.0


In [22]:
predictions3['baseline'] = train.taxvaluedollarcnt.mean()
predictions3.head()

,actual,baseline
13404,849000.0,446248.089772
2305,1298990.0,446248.089772
11329,1636139.0,446248.089772
9365,302961.0,446248.089772
7841,376973.0,446248.089772


In [23]:
x_train_poly_cols = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
x_val_poly_cols = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(x_train_poly_cols)
x_train_poly = pd.DataFrame(
    poly.transform(x_train_poly_cols),
    columns=poly.get_feature_names(x_train_poly_cols.columns),
    index=x_train_poly_cols.index,
)

poly_rm = LinearRegression()
poly_rm.fit(x_train_poly, y_train)

x_validate_poly = poly.transform(x_val_poly_cols)
predictions3['polynomial3_intsonly'] = poly_rm.predict(x_validate_poly)
predictions3.head()

,actual,baseline,polynomial3_intsonly
13404,849000.0,446248.089772,1.065580e+06
2305,1298990.0,446248.089772,1.191486e+06
11329,1636139.0,446248.089772,3.347459e+06
9365,302961.0,446248.089772,4.005164e+05
7841,376973.0,446248.089772,3.979289e+05


In [24]:
x_train_poly_cols = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
x_val_poly_cols = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(x_train_poly_cols)
x_train_poly = pd.DataFrame(
    poly.transform(x_train_poly_cols),
    columns=poly.get_feature_names(x_train_poly_cols.columns),
    index=x_train_poly_cols.index,
)

poly_rm = LinearRegression()
poly_rm.fit(x_train_poly, y_train)

x_validate_poly = poly.transform(x_val_poly_cols)
predictions3['polynomial4_intsonly'] = poly_rm.predict(x_validate_poly)
predictions3.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly
13404,849000.0,446248.089772,1.065580e+06,1.003216e+06
2305,1298990.0,446248.089772,1.191486e+06,1.236203e+06
11329,1636139.0,446248.089772,3.347459e+06,3.313957e+06
9365,302961.0,446248.089772,4.005164e+05,4.581910e+05
7841,376973.0,446248.089772,3.979289e+05,3.771731e+05


In [25]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt']]
lm = LinearRegression()
lm.fit(x, y_train)
predictions3['lm_top2x'] = lm.predict(z)
predictions3.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x
13404,849000.0,446248.089772,1.065580e+06,1.003216e+06,8.833172e+05
2305,1298990.0,446248.089772,1.191486e+06,1.236203e+06,9.607140e+05
11329,1636139.0,446248.089772,3.347459e+06,3.313957e+06,1.819518e+06
9365,302961.0,446248.089772,4.005164e+05,4.581910e+05,4.151755e+05
7841,376973.0,446248.089772,3.979289e+05,3.771731e+05,4.352505e+05


In [26]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt']]
lm2 = LinearRegression()
lm2.fit(x, y_train)
predictions3['lm_top3x'] = lm2.predict(z)
predictions3.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x,lm_top3x
13404,849000.0,446248.089772,1.065580e+06,1.003216e+06,8.833172e+05,8.808659e+05
2305,1298990.0,446248.089772,1.191486e+06,1.236203e+06,9.607140e+05,9.588236e+05
11329,1636139.0,446248.089772,3.347459e+06,3.313957e+06,1.819518e+06,1.817400e+06
9365,302961.0,446248.089772,4.005164e+05,4.581910e+05,4.151755e+05,4.128410e+05
7841,376973.0,446248.089772,3.979289e+05,3.771731e+05,4.352505e+05,4.359318e+05


In [27]:
x = x_train_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
z = x_validate_scaled_df[['calculatedfinishedsquarefeet', 'yearbuilt', 'bathroomcnt', 'bedroomcnt']]
lm3 = LinearRegression()
lm3.fit(x, y_train)
predictions3['lm_allx'] = lm3.predict(z)
predictions3.head()

,actual,baseline,polynomial3_intsonly,polynomial4_intsonly,lm_top2x,lm_top3x,lm_allx
13404,849000.0,446248.089772,1.065580e+06,1.003216e+06,8.833172e+05,8.808659e+05,8.736484e+05
2305,1298990.0,446248.089772,1.191486e+06,1.236203e+06,9.607140e+05,9.588236e+05,1.022328e+06
11329,1636139.0,446248.089772,3.347459e+06,3.313957e+06,1.819518e+06,1.817400e+06,1.907199e+06
9365,302961.0,446248.089772,4.005164e+05,4.581910e+05,4.151755e+05,4.128410e+05,4.592332e+05
7841,376973.0,446248.089772,3.979289e+05,3.771731e+05,4.352505e+05,4.359318e+05,4.018144e+05


In [28]:
def calculate_mse(y_predicted):
    return mean_squared_error(predictions3.actual, y_predicted)

predictions3.apply(calculate_mse)

actual                  0.000000e+00
baseline                3.857902e+11
polynomial3_intsonly    1.826847e+11
polynomial4_intsonly    1.804372e+11
lm_top2x                2.404422e+11
lm_top3x                2.404597e+11
lm_allx                 2.332532e+11
dtype: float64

In [29]:
predictions3.apply(calculate_mse).apply(math.sqrt)

actual                       0.000000
baseline                621120.086941
polynomial3_intsonly    427416.340829
polynomial4_intsonly    424779.037957
lm_top2x                490349.054020
lm_top3x                490366.883189
lm_allx                 482962.909847
dtype: float64